<a href="https://colab.research.google.com/github/derek-shing/DS-Unit-1-Sprint-5-Data-Storytelling-Blog-Post/blob/master/Supreme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Load 1 product info

import requests
import bs4
import pandas as pd
import datetime
import time

url='https://supremedatabase.com/seasons/fall-winter-2018/sweatshirts/box-logo-crewneck'

res = requests.get(url)


In [0]:
res.text
res.content

In [0]:
mysoup = bs4.BeautifulSoup(res.text)

mysoup

In [0]:
def extractinfo(url):
  res = requests.get(url)
  mysoup = bs4.BeautifulSoup(res.text)
  info = mysoup.select('.info')
  type(info[0])

  infohtml = info[0]
  name=infohtml.find('h3').get_text()
  temp=infohtml.find_all('p')
  releasedate=temp[0].get_text()
  price=temp[1].get_text()
  desc=temp[2].get_text()
#infolist=list(infohtml.children)

#infolist.


  releasedate=pd.to_datetime(releasedate, infer_datetime_format=True)
  price=float(price.strip('$').replace(',',''))
  
  info_dict = {'name':name,'price':price,'release_date':releasedate,'desc':desc}
  return info_dict

In [0]:
#loop all item on the same day

url = 'https://supremedatabase.com/seasons/fall-winter-2018/drops/2018-12-06'
res=requests.get(url)


In [0]:
mysoup=bs4.BeautifulSoup(res.text)

In [0]:
def get_drop_info(url):
  items=[]
  res=requests.get(url)
  mysoup=bs4.BeautifulSoup(res.text)
  html=mysoup.select('.items a')
  for item in html:
    url='https://supremedatabase.com'+item['href']
    items.append(extractinfo(url))
    
  return items

#url = 'https://supremedatabase.com/seasons/fall-winter-2018/drops/2018-12-06'
#get_drop_info(url)

In [0]:
#Load different day

def get_release_date_info(url):
  items=[]
  #url = 'https://supremedatabase.com/seasons/fall-winter-2018/drops'
  res=requests.get(url)
  mysoup=bs4.BeautifulSoup(res.text)
  html=mysoup.select('.drops p a')
  for item in html:
    url='https://supremedatabase.com'+item['href']
    items.extend(get_drop_info(url))
    
  return items


In [0]:
product=[]
url='https://supremedatabase.com/seasons/fall-winter-2018/drops'

product.extend(get_release_date_info(url))

In [0]:
product=[]
url='https://supremedatabase.com/seasons/spring-summer-2018/drops'
product.extend(get_release_date_info(url))

In [7]:
len(product)
#product

424

In [0]:
df = pd.DataFrame(product)

In [13]:
df.sort_values(by='price', ascending=False).head()

,desc,name,price,release_date
240,Lambskin leather with poly and nylon lining. F...,Supreme®/The North Face® Leather Mountain Parka,1098.0,2018-10-19
124,Custom fit Schott® Perfecto leather jacket wit...,Supreme®/Comme des Garçons SHIRT® Schott® Pain...,998.0,2018-09-13
333,Sheepskin with full zip closure and D-ring str...,Patchwork Shearling B-3 Jacket,868.0,2018-11-23
36,Lambskin leather with satin lining and half zi...,Leather Anorak,698.0,2018-08-23
267,Diamond quilted lambskin leather with nickel l...,Quilted Studded Leather Jacket,698.0,2018-11-01


In [77]:
df.head()

,desc,name,price,release_date,searchurl
0,All cotton classic Hanes® boxer brief.,Supreme®/Hanes® Boxer Briefs (4 Pack),36.0,2018-08-23,https://stockx.com/search?s=Supreme®/Hanes®%Bo...
1,All cotton 14 oz. overdyed denim crusher with ...,Diamond Stitch Crusher,60.0,2018-08-23,https://stockx.com/search?s=Diamond%Stitch%Cru...
2,All cotton pique with patchwork pattern. On se...,Patchwork Pique Short,118.0,2018-08-23,https://stockx.com/search?s=Patchwork%Pique%Short
3,All cotton 14 oz. denim with enzyme wash. Clas...,Washed Regular Jean,138.0,2018-08-23,https://stockx.com/search?s=Washed%Regular%Jean
4,"2,420mAh protective battery case. Extends iPho...",Supreme®/mophie® iPhone 8 Plus Juice Pack Air,128.0,2018-08-23,https://stockx.com/search?s=Supreme®/mophie®%i...


# LOAD DATA FROM STOCKX

In [0]:
#find current price
!pip install selenium



In [0]:
from selenium import webdriver

In [0]:
browser = webdriver
browser.Firefox()

In [0]:
type(browser)

In [0]:
url='https://stockx.com/search?s=supreme%20box%20logo'
result = requests.get(url)

In [0]:
xsoup=bs4.BeautifulSoup(result.text)

In [0]:
xsoup

In [0]:
!pip install requests-html

In [0]:
from requests_html import HTMLSession

session = HTMLSession()
r = session.get(url)

In [0]:
type(r)


In [0]:
r.html.render()

### Get result page by searching product name

In [19]:
xlist ={}

#product_name= 'Supreme®/Hanes® Boxer Briefs (4 Pack)'#df.name
#product_name.head()
def get_search_result_page(product_name):
  wd.get('https://stockx.com/search?s=')
  xsoup = bs4.BeautifulSoup(wd.page_source)
  #print(xsoup.find(id='root').find(class_='wrapper'))
  search_bar = wd.find_element_by_id("page-search-bar")
  search_bar
  search_bar.send_keys(product_name)
  time.sleep(1)
  return wd.page_source
#print(wd.page_source)

trial=df.sort_values(by='price', ascending=False)


resell_list=[]
loading=0
for product_name in trial.name:
  loading=loading+1
  print(loading)
#for url in df.searchurl:
#url=df.searchurl[0]
#print(url)
#result = requests.get(url)
#type(result)
  s=get_search_result_page(product_name)
#print(s)

  tile_list = get_result_tile(s)
  if len(tile_list)>0:
    for tile in tile_list:
      resell_list.append(extract_resell_price(tile,product_name))
  
resell_list  


#search_box = xsoup.find(id='root').find(class_='wrapper').find(id='site-header').find(class_='page-container vacation-banner-padding').find(id='wrap')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


[{'key': 'Supreme®/RIMOWA Topas Multiwheel® 82L',
  'name': 'Supreme RIMOWA Topas Multiwheel 82L Red',
  'resell_price': 4599.0},
 {'key': 'Supreme®/RIMOWA Topas Multiwheel® 82L',
  'name': 'Supreme RIMOWA Topas Multiwheel 82L Black',
  'resell_price': 3800.0},
 {'key': 'Supreme®/RIMOWA Topas Multiwheel® 45L',
  'name': 'Supreme RIMOWA Topas Multiwheel 45L Red',
  'resell_price': 3960.0},
 {'key': 'Supreme®/RIMOWA Topas Multiwheel® 45L',
  'name': 'Supreme RIMOWA Topas Multiwheel 45L Black',
  'resell_price': 2900.0},
 {'key': 'Skull Pile Leather Bomber Jacket',
  'name': 'Supreme Skull Pile Leather Bomber Jacket Green',
  'resell_price': 450.0},
 {'key': 'Studded Arc Logo Leather Jacket',
  'name': 'Supreme Studded Arc Logo Leather Jacket Black',
  'resell_price': 1000.0},
 {'key': 'Studded Arc Logo Leather Jacket',
  'name': 'Supreme Studded Arc Logo Leather Jacket White',
  'resell_price': 3333.0},
 {'key': 'Studded Arc Logo Leather Jacket',
  'name': 'Supreme Studded Arc Logo Leath

In [0]:
df_x = pd.DataFrame(resell_list)

In [21]:
df_x.head()

,key,name,resell_price
0,Supreme®/RIMOWA Topas Multiwheel® 82L,Supreme RIMOWA Topas Multiwheel 82L Red,4599.0
1,Supreme®/RIMOWA Topas Multiwheel® 82L,Supreme RIMOWA Topas Multiwheel 82L Black,3800.0
2,Supreme®/RIMOWA Topas Multiwheel® 45L,Supreme RIMOWA Topas Multiwheel 45L Red,3960.0
3,Supreme®/RIMOWA Topas Multiwheel® 45L,Supreme RIMOWA Topas Multiwheel 45L Black,2900.0
4,Skull Pile Leather Bomber Jacket,Supreme Skull Pile Leather Bomber Jacket Green,450.0


### Saving web scrap result to CSV

In [0]:
from google.colab import files

df_x.to_csv('resell.csv')
files.download('resell.csv')

In [0]:
df.to_csv('product.csv')
files.download('product.csv')

In [0]:
trial.head(10)

In [2]:

s=get_search_result_page('Wool Trench Coat')
tile_list = get_result_tile(s)
tile_list
for tile in tile_list:
      print(tile)
      resell_list.append(extract_resell_price(tile,product_name))

NameError: ignored

In [83]:
len(tile_list)

0

In [0]:
print(s)

In [0]:
search_box = xsoup.find(id='root').find(class_='wrapper').find(class_='page-container vacation-banner-padding').find(id='wrap')

In [0]:
search_box

In [0]:
search_box = search_box.find(class_='content-header container-fluid hidden-xs sneakers').find(class_='home-jumbo').find(class_='container search').find(id='HomePageSearchBoxWrapper')

In [0]:
search_box = search_box.find(class_='product-search-control').find(class_='search-row').find(class_='search-input').find('form')

In [0]:
search_box=search_box.find('input')

In [60]:
search_box

<input autocapitalize="off" autocomplete="off" autocorrect="off" class="searchbox input-text form-control" id="home-search" name="q" placeholder="Search for brand, color, etc." spellcheck="false" type="search" value=""/>

In [0]:
product_name='Supreme®/Hanes® Boxer Briefs (4 Pack)'#df.name[0]
#type(search_box)
search = wd.find_element_by_id("home-search")


In [0]:
search.send_keys(Keys.ENTER)

In [0]:
search.send_keys(product_name)

In [26]:
type(search)

selenium.webdriver.remote.webelement.WebElement

In [0]:
print(wd.page_source)

### **Try using selenium**

In [0]:
# install chromium, its driver, and selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..



In [14]:

!pip install selenium
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
wd = webdriver.Chrome('chromedriver',options=options)

In [0]:
wd = webdriver.Chrome('chromedriver',options=options)

In [0]:
wd.implicitly_wait(20)
wd.get('https://stockx.com/search?s=supreme')
#wd.get('https://www.google.com')
print(wd.page_source)  # results
type(wd.page_source)



# **Get the result-tile tag**

In [0]:
def get_result_tile(source):
  xsoup = bs4.BeautifulSoup(source)
  tile_list=xsoup.find(id='root').find(class_='wrapper').find(class_='page-container').find(id='search-wrapper').find(class_='search-results-grid').find_all(class_='result-tile')
  return tile_list

In [0]:
xsoup = bs4.BeautifulSoup(new_source)

In [0]:
xsoup

In [0]:
tile_list=xsoup.find(id='root')

In [0]:
tile_list=tile_list.find(class_='wrapper')

In [0]:
tile_list=tile_list.find(class_='page-container')

In [0]:
tile_list=tile_list.find(id='search-wrapper')

In [0]:
tile_list=tile_list.find(class_='search-results-grid')

In [0]:
tile_list=tile_list.find_all(class_='result-tile')

In [0]:
tile_list

# Get Name and ask price for a tile

In [0]:
test=tile_list[0]

In [0]:
name

In [0]:
name=test.find('a').find('h4').get_text()

In [0]:
price=test.find('a').find(class_='bid bid-button-b').find('a').find(class_='stats').find(class_='stat-value stat-small').get_text()

In [0]:
price=float(price.strip('$').replace(',',''))

In [0]:
price

In [0]:
def extract_resell_price(tile,key):
  name=tile.find('a').find('h4').get_text()
  price=tile.find('a').find(class_='bid bid-button-b').find('a').find(class_='stats')
  price=price.findChildren("div")[0].get_text()
  if price=='--':
    price=0
  else:
    price=float(price.strip('$').replace(',',''))
  return {'name':name,'resell_price':price,'key':key}
  

In [38]:
resell_list=[]

#for url in df.searchurl:
#url=df.searchurl[0]
#print(url)
#result = requests.get(url)
#type(result)
tile_list = get_result_tile(wd.page_source)
for tile in tile_list:
    resell_list.append(extract_resell_price(tile,url))
  
resell_list  


[{'key': 'https://supremedatabase.com/seasons/fall-winter-2018/sweatshirts/box-logo-crewneck',
  'name': 'Supreme Hanes Boxer (4 Pack) Briefs Black',
  'resell_price': 54.0},
 {'key': 'https://supremedatabase.com/seasons/fall-winter-2018/sweatshirts/box-logo-crewneck',
  'name': 'Supreme Hanes Boxer (4 Pack) Briefs White',
  'resell_price': 51.0},
 {'key': 'https://supremedatabase.com/seasons/fall-winter-2018/sweatshirts/box-logo-crewneck',
  'name': 'Supreme Hanes Crew Socks Crew Socks (4 Pack) White',
  'resell_price': 28.0},
 {'key': 'https://supremedatabase.com/seasons/fall-winter-2018/sweatshirts/box-logo-crewneck',
  'name': 'Supreme Hanes Socks(4 Pack) Black',
  'resell_price': 38.0}]

In [0]:
stockx_df=pd.DataFrame(resell_list)

In [0]:
stockx_df

### Simulate Scroll down to get more search result html tag

In [0]:
from selenium.webdriver.common.keys import Keys
def scrollDown(browser, numberOfScrollDowns):
 body = browser.find_element_by_tag_name("body")
 while numberOfScrollDowns >=0:
  body.send_keys(Keys.PAGE_DOWN)
  numberOfScrollDowns -= 1
 return browser

In [0]:
import time
SCROLL_PAUSE_TIME = 2

# Get scroll height
last_height = wd.execute_script("return document.body.scrollHeight")
last_source=wd.page_source

print('doc height:',last_height)

#for i in range(2):
while True:
    # Scroll down to bottom
    #wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    wd=scrollDown(wd,10)
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_source =wd.page_source
    if new_source==last_source:
        break
    last_source = new_source
    
print(new_source)

### Remove color from the name and add a feature

In [0]:
color list=[' Grey',' Light Pink',' Red',]

In [51]:
stockx_df.name

,name,resell_price
0,Supreme SteiffBear Heather Grey,249.0
1,Supreme Hanes Boxer (4 Pack) Briefs Black,52.0
2,Supreme ShoulderBag (FW18) Black,105.0
3,Supreme Hanes Boxer (4 Pack) Briefs White,47.0
4,Supreme WaistBag (FW18) Black,101.0


In [0]:
#find highest profit

In [0]:
#find rare item

In [0]:
#recongize some pattern which will have profit protenial

In [0]:
#other company....